In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import time
import argparse
import random
from timm.models.vision_transformer import VisionTransformer


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name, num=10):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, num)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

# -------------------- ResNet20 定义 --------------------
# (ResNet20 定义代码保持不变，此处省略以节省空间)
def _weights_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    def forward(self, x):
        return self.lambd(x)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)
        # Don't apply weight init here if loading pretrained weights
        # self.apply(_weights_init)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride_val in strides: # Renamed variable to avoid conflict
            layers.append(block(self.in_planes, planes, stride_val)) # Use renamed variable
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        # Use adaptive average pooling for robustness
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet20():
    return ResNet(BasicBlock, [3, 3, 3], num_classes=10)
def ResNet32():
    """ ResNet-32 model configuration based on 6n+2 formula """
    # (32-2)/6 = 5
    return ResNet(BasicBlock, [5, 5, 5], num_classes=10)

def ResNet44():
    """ ResNet-44 model configuration based on 6n+2 formula """
    # (44-2)/6 = 7
    return ResNet(BasicBlock, [7, 7, 7], num_classes=10)

def ResNet56():
    """ ResNet-56 model configuration based on 6n+2 formula """
    # (56-2)/6 = 9
    return ResNet(BasicBlock, [9, 9, 9], num_classes=10)

def ResNet110():
    """ ResNet-110 model configuration based on 6n+2 formula """
    # (110-2)/6 = 18
    return ResNet(BasicBlock, [18, 18, 18], num_classes=10)

# Hardcode the values instead:
class Args:
    model = 'r110'
    load_path = './resnet110_cifar10_final.pth'
    ft_epochs = 10
    ft_lr = 0.001  # 0.001
    save_path = './resnet110_cifar10_sgd_ft.pth'
    batch_size = 128
    data_path = './data'
args = Args()


# -------------------- 设备配置 --------------------
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# -------------------- 数据准备 --------------------
print('==> Preparing data..')
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
])

trainset = torchvision.datasets.CIFAR10(
    root=args.data_path, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=args.batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root=args.data_path, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# -------------------- 模型加载 --------------------
print('==> Building and loading pre-trained model..')
# net = ResNet20()
if args.model == 'r20':
    net = ResNet20().to(device)
elif args.model == 'r32':
    net = ResNet32().to(device)
elif args.model == 'r44':
    net = ResNet44().to(device)
elif args.model == 'r56':
    net = ResNet56().to(device)
elif args.model == 'r110':
    net = ResNet110().to(device)
elif args.model == 'vgg16':
    net = VGG('VGG16').to(device)
elif args.model == 'vgg11':
    net = VGG('VGG11').to(device)
elif args.model == 'vit-t':
    net = VisionTransformer(img_size=32,patch_size=4,num_classes=100, embed_dim=192, depth=12, num_heads=3).to(device)
elif args.model == 'vit-s':
    net = VisionTransformer(img_size=32,patch_size=4,num_classes=100, embed_dim=384, depth=12, num_heads=6).to(device)


if os.path.exists(args.load_path):
    try:
        print(f"Loading checkpoint from '{args.load_path}'")
        checkpoint = torch.load(args.load_path, map_location=device)
        # Adjust based on how the model was saved (state_dict vs full model)
        if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
             net.load_state_dict(checkpoint['state_dict'])
        elif isinstance(checkpoint, dict) and not ('state_dict' in checkpoint): # Directly saved state_dict
             net.load_state_dict(checkpoint)
        else: # Saved the entire model object
             net = checkpoint
        print("Pre-trained model loaded successfully.")
    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        print("Proceeding with initialized ResNet20 (training from scratch).")
        # Apply weight initialization if not loading weights
        net.apply(_weights_init)
else:
    print(f"Checkpoint file not found at '{args.load_path}'.")
    print("Proceeding with initialized ResNet20 (training from scratch).")
    # Apply weight initialization if not loading weights
    net.apply(_weights_init)


# Optional DataParallel
# if device == 'cuda' and torch.cuda.device_count() > 1:
#     print(f"Let's use {torch.cuda.device_count()} GPUs!")
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True # Good if input sizes don't change

# -------------------- 损失函数 和 sgd 优化器 --------------------
criterion = nn.CrossEntropyLoss()


# --- 添加 SGD 优化器定义 ---
optimizer = optim.SGD(net.parameters(),       # 使用模型的参数
                      lr=args.ft_lr,          # 使用 fine-tuning 的学习率
                      momentum=0,           # 动量
                      weight_decay=5e-4)      # 权重衰减 (如果需要的话)
# ---------------------------


# No learning rate scheduler needed for fixed LR fine-tuning

# -------------------- 评估函数 (Same as before) --------------------
def evaluate(loader, set_name="Test", model=net): # Pass model explicitly
    model.eval() # Set model to evaluation mode
    eval_loss = 0
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad(): # Disable gradient calculation
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs) # Use the passed model
            loss = criterion(outputs, targets)

            eval_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    epoch_time = time.time() - start_time
    avg_loss = eval_loss / len(loader)
    accuracy = 100. * correct / total
    print(f'{set_name.ljust(5)} | Loss: {avg_loss:.4f} | Acc: {accuracy:.3f}% ({correct}/{total}) | Time: {epoch_time:.2f}s')
    return avg_loss, accuracy


# -------------------- 初始评估 (评估加载的模型) --------------------
print("\n==> Evaluating loaded model before fine-tuning...")
initial_train_loss, initial_train_acc = evaluate(trainloader, "Train", net)
initial_test_loss, initial_test_acc = evaluate(testloader, "Test", net)
print("--------------------------------------------------")


def train_sgd(epoch): # <--- 修改函数名 (或者你喜欢的其他名字)
    print(f'\n--- SGD Fine-tuning Epoch: {epoch+1}/{args.ft_epochs} ---') # 更新打印信息
    net.train() # Set model to training mode
    train_loss = 0
    correct = 0
    total = 0
    start_time = time.time()
    current_lr = optimizer.param_groups[0]['lr'] # Get LR (should be fixed)

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        # --- 标准 SGD 步骤 ---
        optimizer.zero_grad()       # <--- 添加：梯度清零
        outputs = net(inputs)       # <--- 前向传播 (只需要一次)
        loss = criterion(outputs, targets) # <--- 计算损失
        loss.backward()             # <--- 反向传播 (只需要一次)
        optimizer.step()            # <--- 添加：更新权重
        # --- 结束标准 SGD 步骤 ---


        # --- 删除 SAM 相关步骤 ---
        # # 1. First forward/backward pass ...
        # loss.backward()
        # optimizer.first_step(zero_grad=True) # <-- 删除

        # # 2. Second forward/backward pass ...
        # with torch.enable_grad():
        #      criterion(net(inputs), targets).backward() # <-- 删除
        # optimizer.second_step(zero_grad=True) # <-- 删除
        # --- 结束删除 SAM 步骤 ---


        # (损失和准确率的累积部分保持不变)
        train_loss += loss.item()
        _, predicted = outputs.max(1) # Use predictions from the forward pass
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    # (打印 epoch 结果的部分保持不变)
    epoch_time = time.time() - start_time
    epoch_loss = train_loss / len(trainloader)
    epoch_acc = 100. * correct / total
    print(f'Train | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.3f}% ({correct}/{total}) | LR: {current_lr:.5f} | Time: {epoch_time:.2f}s')
    return epoch_loss, epoch_acc

# --- 修改 "Fine-tuning 主循环" ---
print("==> Starting SGD Fine-tuning...") # 更新打印信息
finetuning_start_time = time.time()

for epoch in range(args.ft_epochs):
    # 1. Fine-tune with SGD for one epoch
    # train_loss, train_acc = train_sam(epoch) # 原来的调用
    train_loss, train_acc = train_sgd(epoch) # <--- 使用新的函数名

    # (评估部分保持不变)
    test_loss, test_acc = evaluate(testloader, "Test", net)

total_finetuning_time = time.time() - finetuning_start_time
print(f"\n==> Finished SGD Fine-tuning in {total_finetuning_time:.2f} seconds.") # 更新打印信息


# -------------------- 保存最终 Fine-tuned 模型 --------------------
print(f'==> Saving final fine-tuned model to {args.save_path}')
save_dir = os.path.dirname(args.save_path)
if save_dir and not os.path.exists(save_dir): # Check if save_dir is not empty
    os.makedirs(save_dir)
# Save only the model state_dict is usually preferred
torch.save(net.state_dict(), args.save_path)
# If you need to save optimizer state as well (e.g., to resume SAM training):
# torch.save({
#     'epoch': args.ft_epochs, # Or the actual last epoch number
#     'model_state_dict': net.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(), # Save SAM state
# }, args.save_path)
print("Final fine-tuned model saved.")

# -------------------- 评估最终 Fine-tuned 模型 --------------------
print("\n==> Evaluating final fine-tuned model (after {} epochs)...".format(args.ft_epochs))
print("--- Final Training Set Evaluation ---")
final_train_loss, final_train_acc = evaluate(trainloader, "Train", net)
print("--- Final Test Set Evaluation ---")
final_test_loss, final_test_acc = evaluate(testloader, "Test", net)

print("\n===== Initial Model Performance =====")
print(f"Initial Training Loss: {initial_train_loss:.4f}")
print(f"Initial Training Acc:  {initial_train_acc:.3f}%")
print(f"Initial Test Loss:     {initial_test_loss:.4f}")
print(f"Initial Test Acc:      {initial_test_acc:.3f}%")
print("====================================")

print("\n===== Final Fine-tuned Model Performance =====")
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Final Training Acc:  {final_train_acc:.3f}%")
print(f"Final Test Loss:     {final_test_loss:.4f}")
print(f"Final Test Acc:      {final_test_acc:.3f}%")
print("==========================================")

Using device: cuda:1
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building and loading pre-trained model..
Loading checkpoint from './resnet110_cifar10_final.pth'
Pre-trained model loaded successfully.

==> Evaluating loaded model before fine-tuning...


/tmp/ipykernel_1118339/1081604056.py:222: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.load_path, map_location=device)


Train | Loss: 0.0025 | Acc: 99.988% (49994/50000) | Time: 11.47s
Test  | Loss: 0.2423 | Acc: 94.020% (9402/10000) | Time: 2.77s
--------------------------------------------------
==> Starting SGD Fine-tuning...

--- SGD Fine-tuning Epoch: 1/10 ---
Train | Loss: 0.0047 | Acc: 99.942% (49971/50000) | LR: 0.00100 | Time: 27.89s
Test  | Loss: 0.2448 | Acc: 93.990% (9399/10000) | Time: 2.58s

--- SGD Fine-tuning Epoch: 2/10 ---
Train | Loss: 0.0043 | Acc: 99.944% (49972/50000) | LR: 0.00100 | Time: 28.83s
Test  | Loss: 0.2437 | Acc: 93.970% (9397/10000) | Time: 2.55s

--- SGD Fine-tuning Epoch: 3/10 ---
Train | Loss: 0.0042 | Acc: 99.952% (49976/50000) | LR: 0.00100 | Time: 27.06s
Test  | Loss: 0.2427 | Acc: 93.880% (9388/10000) | Time: 2.45s

--- SGD Fine-tuning Epoch: 4/10 ---
Train | Loss: 0.0041 | Acc: 99.960% (49980/50000) | LR: 0.00100 | Time: 27.55s
Test  | Loss: 0.2423 | Acc: 93.960% (9396/10000) | Time: 2.50s

--- SGD Fine-tuning Epoch: 5/10 ---
Train | Loss: 0.0044 | Acc: 99.962% 